# Branch & Bound algorithm

## Implementation

In [1]:
# Modules that we need.
import numpy as np
import math
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
# Print the tableau matrix.
def pretty_print_tableau(tableau):
    rows = []
    for row in tableau:
        formatted_row = ["{:>6}".format(str(element)) for element in row]
        rows.append("[ " + " ".join(formatted_row) + "]")
    print("\n".join(rows))

In [3]:
# Pivot operations.
def pivot(tableau, n, m, t, h, minipivot=False, row=0):
    save = tableau[t, h]
    for j in range(n + 1):
        tableau[t, j] = tableau[t, j] / save
    if not minipivot:
        for i in range(m + 1):
            if i != t and tableau[i, h] != 0:
                save = tableau[i, h]
                for j in range(n + 1):
                    tableau[i, j] = tableau[i, j] - save * tableau[t, j]
    else:
        if row != t and tableau[row, h] != 0:
            save = tableau[row, h]
            for j in range(n + 1):
                tableau[row, j] = tableau[row, j] - save * tableau[t, j]

In [4]:
# Dual Simplex algorithm.
def dual_simplex(tableau, n, m, beta, x_opt=None, verbose=False):
    unbounded = False
    optimal = False

    itr = 0 # Num of iteration.

    while not optimal and not unbounded:
        if verbose:
            print(f"Current tableau - Itr: {itr}")
            pretty_print_tableau(tableau)

        candidates = [] # Save all the basic variables < 0.
        # Optimality check.
        optimal = True
        for i in range(1, m + 1):
            if tableau[i, 0] < 0:
                optimal = False
                candidates.append(i)

        if not optimal:
            # Choose the var that leaves that basis, according to Bland's rule
            candidates = np.array(candidates)
            t = candidates[0]
            for c in candidates[1:]:
                if beta[c-1] < beta[t]:
                    t = c
                    
            if verbose:
                print(f"x[{beta[t-1]}] leaves the basis.")

            # Unbounded check.
            unbounded = True
            for j in range(1, n + 1):
                if tableau[t, j] < 0:
                    unbounded = False
                    break
            if not unbounded:
                cj_over_aj = np.zeros(n) # Stores the candidates values.
                for j in range(1, n + 1):
                    if tableau[t, j] < 0:
                        cj_over_aj[j-1] = tableau[0, j] / abs(tableau[t, j])
                    else:
                        cj_over_aj[j-1] = float('inf') # Otherwise i can't use 
                                                       # the index returned 
                                                       # by argmin in the tableau.
                                                       # (Sort of placeholder)
                
                # Choose the columns that enters the basis according to Bland's rule.
                h = np.argmin(cj_over_aj)
                h += 1 # Need this to access the corresponding cols in the tableau.

                if verbose:
                    print("Current pivot element =", tableau[t, h])

                pivot(tableau, n, m, t, h) # Update the tableau.

                if verbose:
                    print(f"x[{h}] enters the basis.")
                    print()
                beta[t-1] = h # Update the basis.
                itr += 1

    # Print the optimal solution, if necessary.
    if optimal:
        if verbose:
            print("Optimal solution:")
            print(f"x_B = {[f"x[{i}]" for i in beta]} = {[str(tableau[i, 0]) for i in range(1, m + 1)]}.")
        # Save optimal solution, if necessary.
        if x_opt is not None:
            for i, v in zip(beta, [tableau[i, 0] for i in range(1, m + 1)]):
                x_opt[i-1] = v
        minus_bar_c0 = -1 * tableau[0, 0]
        if verbose:
            print(f"Optimal value: {minus_bar_c0}")
        return minus_bar_c0

    if verbose:
        print("The problem is unbounded.")
    return float('-inf')

In [5]:
# Phase 1 of the simplex method: find a feasible basis.
def phase1(tableau, n, m, beta, verbose=False):
    original_row0 = tableau[0, :] # Save the original row0.

    # Add to the tableau the identity matrix (y_i variables) and the row0.
    id = np.identity(m, dtype=object)
    for i in range(m):
        for j in range(m):
            id[i, j] = Fraction(id[i, j])
    id = np.vstack(([Fraction(1) for _ in range(m)], id))
    tableau = np.concatenate((tableau, id), axis=1)
    for j in range(n + 1):
        tableau[0, j] = Fraction(0)


    # Update the tableau to transform it in his canonical form.
    for i in range(1, m + 1):
        pivot(tableau, n + m, m, i, n + i, minipivot=True)

    # Solve the artificial problem.
    for i in range(m):
        beta.append(n + i + 1)
    cost = simplex(tableau, n + m, m, beta, verbose=verbose)

    # Check cost value.
    if cost != 0:
        print("The original problem is infeasible")
        return None

    # Check degeneracy cases.
    for i, var in enumerate(beta):
        if var > n:
            print(f"Degeneracy found: variable x[{var}].")
            remove_line = True
            # Find the value of the first bar_aij != 0
            for j in range(1, n + 1):
                if tableau[i+1, j] != 0:
                    pivot(tableau, n + m, m, i+1, j)
                    beta[i] = j
                    print(f"x[{j}] enters and x[{var}] leaves the basis.")
                    remove_line = False # No need to remove the line (A is full rank)
                    break
            if remove_line: # A is not full rank.
                tableau = np.delete(tableau, i+1)


    # Remove the y_i cols and restore the original row0.
    tableau = np.delete(tableau, [n + i + 1 for i in range(m)], axis=1)
    tableau[0, :] = original_row0

    # Update the tableau to transform it in his canonical form.
    for i in range(1, m + 1):
        #print(i, beta[i-1]) # TODO: ERROR: the beta vector must be updated.
        pivot(tableau, n, m, i, beta[i - 1], minipivot=True)

    # Proceed with PHASE 2.
    return tableau

In [6]:
# Simplex Method. This implementation uses the Bland's rule.
def simplex(tableau, n, m, beta, x_opt=None, verbose=False):
    unbounded = False
    optimal = False

    itr = 0 # Num of iteration.

    while not optimal and not unbounded:
        if verbose:
            print(f"Current tableau - Itr: {itr}")
            pretty_print_tableau(tableau)

        # Optimality test.
        h = -1 # Index of the column that enters the basis.
        optimal = True
        for j in range(1, n + 1):
            if tableau[0, j] < 0:
                optimal = False
                h = j # Choose the var that eneters the basis (Bland's rule).
                if verbose:
                    print(f"x[{h}] enters the basis.")
                break
        if not optimal:
            # Unbounded check.
            unbounded = True
            for i in range(1, m + 1):
                if tableau[i, h] >= 0:
                    unbounded = False
                    break
            if not unbounded:
                bi_over_ai = np.zeros(m) # Stores the theta candidates.
                for i in range(1, m + 1):
                    if tableau[i, h] > 0:
                        bi_over_ai[i-1] = tableau[i, 0] / tableau[i, h]
                    else:
                        bi_over_ai[i-1] = float('inf') # Otherwise i can't use 
                                                       # the index returned 
                                                       # by argmin in the tableau.
                                                       # (Sort of placeholder)
                
                # Choose the columns that leaves the basis according to Bland's rule.
                candidates, = np.nonzero(bi_over_ai == bi_over_ai.min())
                t = candidates[0]
                for c in candidates[1:]:
                    if beta[c] < beta[t]:
                        t = c
                t += 1 # Need this to access the corresponding row in the tableau.

                if verbose:
                    print("Current pivot element =", tableau[t, h])

                pivot(tableau, n, m, t, h) # Update the tableau.

                if verbose:
                    print(f"x[{beta[t-1]}] leaves the basis.")
                beta[t-1] = h # Update the basis.
                itr += 1
                print()

    # Print the optimal solution, if necessary.
    if optimal:
        if verbose:
            print("Optimal solution:")
            print(f"x_B = {[f"x[{i}]" for i in beta]} = {[str(tableau[i, 0]) for i in range(1, m + 1)]}.")
        minus_bar_c0 = -1 * tableau[0, 0]
        # Save solution, if necessary.
        if x_opt is not None:
            for i, v in zip(beta, [tableau[i, 0] for i in range(1, m + 1)]):
                x_opt[i-1] = v
        if verbose:
            print(f"Optimal value: {minus_bar_c0}")
        return minus_bar_c0

    if verbose:
        print("The problem is unbounded.")
    return float('-inf')

In [7]:
# Check if solution x is integer feasible.
def check_integrality(x, fracidx=None):
    f = True
    for i, v in enumerate(x):
        if not v.is_integer():
            f = False
            if fracidx is not None:
                fracidx.append(i)
    return f

In [8]:
# Branch & Bound algorithm.
def branch_and_bound(A, b, c, max_itr=float("inf"), verbose=False):
    m, n = A.shape # Number of rows and columns

    # Init the tableau in his original form.
    tab = np.array(np.insert(c, 0, Fraction(0))) # Add row0.
    tab = np.vstack((tab, np.column_stack((b, A)))) # Add b and A.

    # Last node of the branching tree.
    i = 0 

    # parent[t] = (+p or -p) where p is the partend 
    #             node of t in the branching tree
    parent = [i]

    # Queue of active open nodes.
    Q = []
    tableau = [tab]
    beta = []

    # Cost of the incumbent and incumbent.
    z_opt = float("inf")
    x_opt = []

    # Solve the linear relaxation of the root and save the 
    # solution cost (lower bound).
    if verbose:
        print("\n### Solve the linear relaxation - PHASE 1 ###")
    beta.append([]) # For the basic variable.
    tableau[0] = phase1(tableau[0], n, m, beta[0], verbose=verbose)
    if tableau[0] is None:
        return

    if verbose:
        print("\n### Solve the linear relaxation - PHASE 2 ###")
    x = [0] * (tableau[0].shape[1] - 1) # Relaxation opt solution.
    LB = [simplex(tableau[0], n, m, beta[0], x_opt=x, verbose=verbose)]
    if float("-inf") == LB[0]:
        return

    # beta is now a list of np.array.
    beta[0] = np.array(beta[0])

    # Check if the solution is integer feasible.
    fracidx = [] # Store fractional indices.
    if check_integrality(x, fracidx=fracidx) and LB[0] < z_opt:
        z_opt = LB[0] # The optimal solution of the problem.
        print("z_opt =", z_opt)

    # vbranch[t] = index h of the branching variable x_h.
    vbranch = [] 
    # value[t] = value x*_h of the branching variable.
    value = []

    # Branching (relaxation solution is not integer).
    if LB[0] < z_opt:
        # Choose a fractional branching variable x*_h
        h = fracidx[0] + 1# (Choose the first index).
        vbranch.append(h) # Save branching var inxex.
        value.append(x[fracidx[0]]) # Save branching var value.
        Q.append(i) # Add the root to the open acrive notes.

    # Process all the acrive nodes.
    itr = 0
    while len(Q) and itr <= max_itr:
        # Choose node t in Q to process.
        # Lets use BFS for now.
        t = Q.pop(0) # i.e. FIFO.
        
        # Define index and value of branching variable.
        h = vbranch[t]
        val = value[t]

        # Generate the child of node t.
        for child in range(1, 3):
            i += 1 # New last node of the branching tree.
            if child == 1:
                parent.append(t)
            else:
                parent.append(-t)

            # Child tableau and basis.
            child_tab = tableau[t]
            child_beta = beta[t]


            # First compute the values on the new row.
            new_row = np.zeros(child_tab.shape[1] + 1, dtype=object)
            # Last entry is +1 for left child, -1 for right.
            new_row[-1] = 1 if child == 1 else -1 
            # First entry is floor(x*_h) for left child, ceil(x*_h) for right.
            new_row[0] = math.floor(val) if child == 1 else math.ceil(val)
            new_row[h] = 1
            
            # Add a new column to the tableau.
            child_tab = np.column_stack((child_tab, [0] * child_tab.shape[0]))
            # Add the new row to the tableau.
            child_tab = np.vstack((child_tab, new_row))

            # Minipivot operations.
            # Find the pivot element.
            row = -1
            for idx, v in enumerate(child_tab[:, h]):
                if v == 1:
                    row = idx
                    break
            # Left child need only a sinlge minipivot. 
            pivot(child_tab, child_tab.shape[1]-1, child_tab.shape[0]-1, row, h, minipivot=True, row=child_tab.shape[0]-1)
            # Right child need also the second minipivot.
            if child == 2:
                pivot(child_tab, child_tab.shape[1]-1, child_tab.shape[0]-1, child_tab.shape[0]-1, child_tab.shape[1]-1, minipivot=True, row=child_tab.shape[0]-1)
            # Update child basis: add the new slack variable.
            child_beta = np.append(child_beta, child_tab.shape[1] - 1)
            
            # Solve the the child problem usgin the dual simplex.
            if verbose:
                print(f"\n### Solving child {i} ###")
            # Space for child solution.
            x = [0] * (child_tab.shape[1] - 1)
            LB.append(dual_simplex(child_tab, child_tab.shape[1]-1, child_tab.shape[0]-1, child_beta, x_opt=x, verbose=verbose))

            # Update the incumbent if necessary.
            fracidx = [] # Store fractional indices.
            if LB[-1] != float("-inf") and check_integrality(x, fracidx=fracidx) and LB[-1] < z_opt:
                x_opt = x
                z_opt = LB[-1]

                # Also prune by optimality, if necessary.
                for idx, q in enumerate(Q):
                    if LB[q] >= LB[-1]:
                        Q.pop(idx)

            # Branch, if necessary.
            if LB[-1] != float("-inf") and LB[-1] < z_opt:
                # Choose a fractional branching variable x*_k
                k = fracidx[0] + 1 # (Choose the first index).
                vbranch.append(k) # Save branching var inxex.
                value.append(x[fracidx[0]]) # Save branching var value.
                tableau.append(child_tab) # Save tableau of the child.
                beta.append(child_beta) # Save basis of the child.
                Q.append(i) # Add the child node to the open active nodes.

            # (else): prune by infeasibility.
            # Add placeholders.
            else:
                vbranch.append(-1)
                value.append(-1)
                tableau.append(-1)
                beta.append(-1)
                
            itr += 1

    return z_opt, x_opt

In [9]:
# Define a simple problem.
A = np.array(
    [[1, 1, 1, 0, 0],
     [1, 4, 0, -1, 0],
     [8, -8, 0, 0, -1]
    ], dtype=object)

b = np.array([4, 6, 3], dtype=object)
c = np.array([1, 2, 0, 0, 0], dtype=object)

# Transform all in Fraction obj.
to_fraction = lambda x: Fraction(x)
tf_vec = np.vectorize(to_fraction)
A = tf_vec(A)
b = tf_vec(b)
c = tf_vec(c)

## Define a problem

In [10]:
# Define a simple problem.
A = np.array(
    [[2, 2, 1, 0],
     [3, 1, 0, 1,],
    ], dtype=object)

b = np.array([9, 11], dtype=object)
c = np.array([-5, -2, 0, 0], dtype=object)

# Transform all in Fraction obj.
to_fraction = lambda x: Fraction(x)
tf_vec = np.vectorize(to_fraction)
A = tf_vec(A)
b = tf_vec(b)
c = tf_vec(c)

In [11]:
# Solve the defined problem.
z_opt, x_opt = branch_and_bound(A, b, c, verbose=True)
print("\n\nOptimal solution:")
print("z_opt =", z_opt)
print("x_opt =", [str(v) for v in x_opt])


### Solve the linear relaxation - PHASE 1 ###
Current tableau - Itr: 0
[    -20     -5     -3     -1     -1      0      0]
[      9      2      2      1      0      1      0]
[     11      3      1      0      1      0      1]
x[1] enters the basis.
Current pivot element = 3
x[6] leaves the basis.

Current tableau - Itr: 1
[   -5/3      0   -4/3     -1    2/3      0    5/3]
[    5/3      0    4/3      1   -2/3      1   -2/3]
[   11/3      1    1/3      0    1/3      0    1/3]
x[2] enters the basis.
Current pivot element = 4/3
x[5] leaves the basis.

Current tableau - Itr: 2
[      0      0      0      0      0      1      1]
[    5/4      0      1    3/4   -1/2    3/4   -1/2]
[   13/4      1      0   -1/4    1/2   -1/4    1/2]
Optimal solution:
x_B = ['x[2]', 'x[1]'] = ['5/4', '13/4'].
Optimal value: 0

### Solve the linear relaxation - PHASE 2 ###
Current tableau - Itr: 0
[   75/4      0      0    1/4    3/2]
[    5/4      0      1    3/4   -1/2]
[   13/4      1      0   -1/4    1/2]